In [36]:
import numpy as np
import pandas as pd
import chardet

In [37]:
with open('/kaggle/input/sms-spam-collection-dataset/spam.csv', 'rb') as f:
    enc = chardet.detect(f.read()) 

In [38]:
dataset=pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding=enc['encoding'])

In [39]:
Y=dataset['v1'].to_numpy()
X=dataset['v2'].to_numpy()

In [40]:
X.shape,Y.shape

((5572,), (5572,))

In [41]:
dataset = np.array([X,Y])

In [42]:
dataset.shape

(2, 5572)

In [43]:
samples=len(dataset[1])
x_train = dataset[0,0:int(0.7*samples)+1]
y_train = dataset[1,0:int(0.7*samples)+1]

x_test = dataset[0,int(0.7*samples)+1:samples+1]
y_test = dataset[1,int(0.7*samples)+1:samples+1]

In [44]:
len(x_test)+len(x_train)

5572

In [45]:
def create_dict(x_train,x_test,y_train,y_test):
    total_words=[]
    spam_words=[]
    ham_words=[]
    for i in range(len(x_train)):
        total_words=total_words+x_train[i].split(" ")
        if(y_train[i]=='spam'):
            spam_words=spam_words + x_train[i].split(" ")
        else:
            ham_words=ham_words + x_train[i].split(" ")
    total_words_set = set(total_words)
    spam_words_set = set(spam_words)
    ham_words_set = set(ham_words)
    
    total_words_dict = {}
    spam_words_dict = {}
    ham_words_dict = {}
    for i in total_words:
        total_words_dict[i] = total_words.count(i)
    for i in spam_words:
        spam_words_dict[i] = spam_words.count(i)
    for i in ham_words:
        ham_words_dict[i] = ham_words.count(i)
    return total_words_dict,spam_words_dict,ham_words_dict,total_words,spam_words,ham_words

In [46]:
def _spam(doc,spam_words_dict,spam_words_list,total_words_dict,total_words_list):
    c=d=1
    for i in doc.split(" "):
        if(i in spam_words_list):
            c*=spam_words_dict[i]/len(spam_words_list)
            d*=total_words_dict[i]/len(total_words_list)
        if(d==0):
            c+=1
            d+=1
        c*=len(spam_words_list)/len(total_words_list)
        return c/d

In [47]:
def _ham(doc,ham_words_dict,ham_words_list,total_words_dict,total_words_list):
    c=d=1
    for i in doc.split(" "):
        if(i in ham_words_list):
            c*=ham_words_dict[i]/len(ham_words_list)
            d*=total_words_dict[i]/len(total_words_list)
        if(d==0):
            c+=1
            d+=1
        c*=len(ham_words_list)/len(total_words_list)
        return c/d

In [48]:
def predict(doc,ham_dict,spam_words_dict,ham_words_list,spam_words_list,total_words_list,total_words_dict):
    prob_spam_ = _spam(doc,spam_words_dict,spam_words_list,total_words_dict,total_words_list)
    prob_ham_ = _ham(doc,ham_words_dict,ham_words_list,total_words_dict,total_words_list)
    if(prob_spam_>prob_ham_):
        return 'spam'
    return 'ham'

In [49]:
def accuracy(x_test, y_test, total_words_dict, spam_dict, ham_dict, total_words_list, spam_words_list, ham_words_list):
    count=0
    for i in range(len(y_test)):
        p = predict(x_test[i],ham_words_dict,spam_words_dict,ham_words_list,spam_words_list,total_words_list,total_words_dict)
        if(y_test[i]==p):
            count+=1
    return (count/len(y_test))*100

In [50]:
total_words_dict,spam_words_dict,ham_words_dict,total_words_list,spam_words_list,ham_words_list = create_dict(x_train,y_train,y_train,y_test)

In [51]:
accuracy(x_test,y_test,total_words_dict,spam_words_dict,ham_words_dict,total_words_list,spam_words_list,ham_words_list)

91.14302812687013